# Scientific Computing 6.2: Parameter sensitivity
This program returns to the Lotka-Volterra predator-prey model, given by the following equations:

$$
\frac{{dR}}{{dt}} = \alpha R(t) - \beta R(t)F(t), \quad \frac{{dF}}{{dt}} = -\gamma F(t) + \delta R(t)F(t)
$$

where:
- $R(t)$: population of rabbits (the prey) over time
- $F(t)$: population of foxes (the predators) over time

The parameters are:
- $\alpha$: reproduction rate of rabbits
- $\beta$: rate at which foxes kill rabbits
- $\gamma$: fox mortality rate
- $\delta$: reproduction rate of foxes relative to rabbit abundance

In [81]:
import numpy as np
import matplotlib.pyplot as plt

### Parameter sensitivity analysis
Now we want to determine the sensitivity of the model to small changes in the initial parameters. To do this, we solve the full transient initial value problem to find the evolution of the populations of rabbits and foxes using the following parameters:
- $\alpha = 100$
- $\beta = 1.5$
- $\gamma = 10$
- $\delta = 0.01$

From the initial condition $R(0) = 1000$, $F(0) = 100$, this program solves the Lotka–Volterra equations numerically from $t = 0$ to $t = 1$, using the Runge-Kutta 4 method. Finally, the relative condition numbers $k_{rel}$ is determined for variations in each parameter (e.g. $\alpha = \alpha + k \alpha$). The corresponding relative condition is given by:

$$k_{rel,\alpha} = \left| \frac{f(\alpha + k \alpha, \beta, \gamma, \delta)}{f(\alpha, \beta, \gamma, \delta)} -1 \right| k^{-1}$$

### Initialization

In [82]:
alpha = 100  # rabbit reproduction rate
beta = 1.5  # fox kill rate
gamma = 10  # fox death rate
delta = 0.01  # relative fox reproduction rate

k = 0.0001  # relative variation in the parameters (parameter * (1+k))

T = 1  # total simulation time
h = 0.002  # step size
N = int(np.ceil(T/h))  # number of time steps

x = np.array([[1000], [100]])  # initial rabbit and fox population
t = h * np.arange(N+1)  # vector of time steps

### Functions

In [83]:
def f(x, a, b, g, d):  # function that returns function value of Lokta-Volterra system
    return np.array([[a*x[0, 0] - b*x[0, 0]*x[1, 0]], [d*x[0, 0]*x[1, 0] - g*x[1, 0]]])


def runge_kutta_4(x, h, N, a, b, g, d):
    result = np.zeros((x.shape[0], N+1))  # pre-allocate result array
    result[:, 0] = x[:, 0]  # store the initial state in the result array

    for i in range(N):
        k1 = f(x, a, b, g, d)
        k2 = f(x + k1 * h/2, a, b, g, d)
        k3 = f(x + k2 * h/2, a, b, g, d)
        k4 = f(x + k3 * h, a, b, g, d)

        x_new = x + h/6 * (k1 + 2*k2 + 2*k3 + k4)
        result[:, i+1] = x_new[:, 0]  # store the new state in the result array
        x = x_new

    return result


def sensitivity(x, h, N, a, b, g, d, k):
    # We are interested in the foxes value at t = 1 (take last element of fox result):
    func = runge_kutta_4(x, h, N, a, b, g, d)[1, -1]
    func_a = runge_kutta_4(x, h, N, (1+k)*a, b, g, d)[1, -1]
    func_b = runge_kutta_4(x, h, N, a, (1+k)*b, g, d)[1, -1]
    func_g = runge_kutta_4(x, h, N, a, b, (1+k)*g, d)[1, -1]
    func_d = runge_kutta_4(x, h, N, a, b, g, (1+k)*d)[1, -1]

    # Calculate relative condition numbers:
    kappa_a = abs(func_a/func - 1) / k
    kappa_b = abs(func_b/func - 1) / k
    kappa_g = abs(func_g/func - 1) / k
    kappa_d = abs(func_d/func - 1) / k

    return kappa_a, kappa_b, kappa_g, kappa_d

### Main code

In [84]:
# Get relative condition numbers and print results:
k_a, k_b, k_g, k_d = sensitivity(x, h, N, alpha, beta, gamma, delta, k)

print(f"kappa_alpha = {k_a:.4f}")
print(f"kappa_beta  = {k_b:.4f}")
print(f"kappa_gamma = {k_g:.4f}")
print(f"kappa_delta = {k_d:.4f}")

kappa_alpha = 1.1724
kappa_beta  = 1.6756
kappa_gamma = 0.7682
kappa_delta = 0.0254


The parameter $\beta$ (fox kill rate) is the most sensitive, as its relative condition number $k_{rel, \beta}$ is the highest. This parameter needs to be known with the highest accuracy to get an accurate model.